# CC3001 Otoño 2020 Tarea 5
# Simulación con Colas y Colas de Prioridad

Profesores: Sección 1 Patricio Poblete / Sección 2 Nelson Baloian

Plazo: Lunes 22 de junio de 2020 a las 23:59

Estudiante: Pía Contreras Guerrero

# A lo que se quiere llegar P1

In [48]:
import numpy as np
class Cola:
    def __init__(self):
        self.q=[]
    def enq(self,x): #encolar, meter x a la cola
        self.q.insert(0,x)
    def deq(self): #saca el primer elemento de la cola
        assert len(self.q)>0
        return self.q.pop()
    def is_empty(self): #ve si la cola está vacía
        return len(self.q)==0
    def size(self): #tamaño de la cola
        return len(self.q)

def print_list(L):
    for x in L:
        print(x)

def simula(maxclientes,ta,ts,debug=False):
    # simula el paso de maxclientes clientes por el sistema,
    # ta = tiempo promedio entre llegadas (arrivals)
    # ts = tiempo promedio de servicio
    # maxclientes = número maximo de clientes para la simulacion
    # debug = True => genera bitácora y usa secuencia aleatoria reproducible
    # retorna (largo promedio de la cola, tiempo promedio en el sistema)

    if debug:
        np.random.seed(1234)
    ahora=0 # tiempo simulado
    nclientes=0 # número de clientes que han llegado
    c=Cola()
    evento_llegada=np.random.exponential(ta)
    evento_salida=None
    acum_largo_cola=0 # acumula largo de la cola para sacar promedio al final
    acum_tiempo_en_sistema=0 # acumula tiempos en el sistema para sacar promedio al final
    bitacora=[] # En caso que se pida debug
    
    while evento_llegada is not None or evento_salida is not None:
        # Ver si el próximo evento que toca que ocurra es la llegada o la salida de un cliente
        if (evento_salida is None) or (evento_llegada is not None and (evento_llegada<=evento_salida)):
            # Llega un cliente, avanzamos el tiempo simulado a la hora en que llega
            ahora=evento_llegada
            nclientes+=1
            
            if debug:
                bitacora.append("T="+str(ahora)+
                                " evento_llegada="+str(evento_llegada)+" evento_salida="+str(evento_salida)+
                                " => LLEGADA"+
                                "\n\tcola="+str(c.q))
            # El cliente que acaba de llegar se pone a la cola
            c.enq(ahora) # Basta anotar su hora de llegada
            acum_largo_cola+=c.size()
            
            if c.size()==1: # la cola estaba vacía, lo empezamos a atender de inmediato
                evento_salida=ahora+np.random.exponential(ts) # programamos su hora de salida
            
            if nclientes<maxclientes: # programamos la hora de llegada del siguiente cliente
                evento_llegada=ahora+np.random.exponential(ta)
            else:
                evento_llegada=None
        else:               
            # Termina de atenderse un cliente, avanzamos el tiempo simulado a la hora en que sale
            ahora=evento_salida
            if debug:
                bitacora.append("T="+str(ahora)+
                                " evento_llegada="+str(evento_llegada)+" evento_salida="+str(evento_salida)+
                                " => SALIDA"+
                                "\n\tcola="+str(c.q))
            tllegada=c.deq() # sacamos al cliente de la cola
            acum_tiempo_en_sistema+=(ahora-tllegada)
            
            if c.is_empty(): # no hay nadie en la cola
                evento_salida=None
            else: # Hay un cliente esperando, empezamos a atenderlo
                evento_salida=ahora+np.random.exponential(ts) # programamos su hora de salida
    
    return (acum_largo_cola/maxclientes,acum_tiempo_en_sistema/maxclientes,bitacora)           

In [49]:
(cola,demora,bitacora)=simula(10,100,75,debug=True)
print("Largo promedio de la cola=",cola," Tiempo promedio en el sistema=",demora)

Largo promedio de la cola= 1.9  Tiempo promedio en el sistema= 138.02762271192722


In [50]:
print_list(bitacora)

T=21.25986576184801 evento_llegada=21.25986576184801 evento_salida=None => LLEGADA
	cola=[]
T=78.83677538497456 evento_llegada=78.83677538497456 evento_salida=94.24603167524747 => LLEGADA
	cola=[21.25986576184801]
T=94.24603167524747 evento_llegada=232.71542283099382 evento_salida=94.24603167524747 => SALIDA
	cola=[78.83677538497456, 21.25986576184801]
T=207.7973648436669 evento_llegada=232.71542283099382 evento_salida=207.7973648436669 => SALIDA
	cola=[78.83677538497456]
T=232.71542283099382 evento_llegada=232.71542283099382 evento_salida=None => LLEGADA
	cola=[]
T=256.5855663702635 evento_llegada=265.07595569718336 evento_salida=256.5855663702635 => SALIDA
	cola=[232.71542283099382]
T=265.07595569718336 evento_llegada=265.07595569718336 evento_salida=None => LLEGADA
	cola=[]
T=386.489172340086 evento_llegada=582.4168679105603 evento_salida=386.489172340086 => SALIDA
	cola=[265.07595569718336]
T=582.4168679105603 evento_llegada=582.4168679105603 evento_salida=None => LLEGADA
	cola=[]


## Lo que usted tiene que hacer

### 1. Mejorar este programa introduciendo una cola de prioridad



In [51]:
import numpy as np
def trepar(a,j): # El elemento a[j] trepa hasta su nivel de prioridad 
    while j>=1 and a[j][0]<a[(j-1)//2][0]: #compara elemento 0 de la tupla, que es el tiempo, pero mueve la tupla completa
        (a[j],a[(j-1)//2])=(a[(j-1)//2],a[j]) # intercambiamos con el padre
        j=(j-1)//2 # subimos al lugar del padre
        
def hundir(a,j,n): # El elemento a[j] se hunde hasta su nivel de prioridad
    while 2*j+1<n: # mientras tenga al menos 1 hijo
        k=2*j+1 # el hijo izquierdo
        if k+1<n and a[k+1][0]<a[k][0]: # el hijo derecho existe y es menor
            k+=1
        if a[j][0]<=a[k][0]: # tiene mejor prioridad que ambos hijos
            break
        (a[j],a[k])=(a[k],a[j]) # se intercambia con el menor de los hijos
        j=k # bajamos al lugar del menor de los hijos
    
class Heap:
    def __init__(self,maxn=100):
        self.a=np.zeros(maxn, dtype="object")
        self.n=0
    def insert(self,x):
        assert self.n<len(self.a)
        self.a[self.n]=x    
        trepar(self.a,self.n)
        self.n+=1       
    def extract_min(self):
        x=self.a[0] # esta variable lleva el mínimo, el casillero 0 queda vacante
        self.n-=1   # achicamos el heap
        self.a[0]=self.a[self.n] # movemos el elemento sobrante hacia el casillero vacante
        hundir(self.a,0,self.n)
        return x
    def imprimir(self):
        print(self.a[0:self.n])
    def size(self): #tamaño de la cola
        return self.n
    def is_empty(self):
        return self.a is None

class Cola:
    def __init__(self):
        self.q=[]
    def enq(self,x):
        self.q.insert(0,x)
    def deq(self):
        assert len(self.q)>0
        return self.q.pop()
    def is_empty(self):
        return len(self.q)==0
    def size(self):
        return len(self.q)

In [52]:
def print_list(L):
    for x in L:
        print(x)

def simula(maxclientes,ta,ts,debug=False):
    # simula el paso de maxclientes clientes por el sistema,
    # ta = tiempo promedio entre llegadas (arrivals)
    # ts = tiempo promedio de servicio
    # maxclientes = número maximo de clientes para la simulacion
    # debug = True => genera bitácora y usa secuencia aleatoria reproducible
    # retorna (largo promedio de la cola, tiempo promedio en el sistema)

    if debug:
        np.random.seed(1234)
    ahora=0 # tiempo simulado
    nclientes=0 # número de clientes que han llegado
    H=Heap(maxclientes)
    c=Cola()
    acum_largo_cola=0 # acumula largo de la cola para sacar promedio al final
    acum_tiempo_en_sistema=0 # acumula tiempos en el sistema para sacar promedio al final
    bitacora=[] # En caso que se pida debug

    #tal como dice la sugerencia, se inserta el primer evento en el Heap
    tipo_evento="evento_llegada" #el primer evento es evento llegada
    tiempo_evento=np.random.exponential(ta) #su tiempo correspondiente es t_a

    H.insert((tiempo_evento,tipo_evento)) #inserta el primer elemento

    while tipo_evento is not None: #se ve que la variable tipo_evento sea distinto a None en cada iteración

        evento=H.extract_min() #se saca el primer elemento y se procesa, como es una tupla
        tiempo_evento=evento[0] #el primer valor es el tiempo
        tipo_evento=evento[1] #el segundo valor es el tipo evento

        # Ver si el próximo evento que toca que ocurra es la llegada o la salida de un cliente
        if tipo_evento=="evento_llegada":
            ahora=tiempo_evento
            # Llega un cliente, avanzamos el tiempo simulado a la hora en que llega
            nclientes+=1
            
            if debug:
                bitacora.append("T="+str(ahora)+
                                " Tiempo="+str(tiempo_evento)+" Tipo evento="+str(tipo_evento)+
                                " => LLEGADA"+
                                "\n\tcola="+str(c.q))
            # El cliente que acaba de llegar se pone a la cola
            c.enq(tiempo_evento) #basta con anotar su hora de llegada
            acum_largo_cola+=c.size()
            if c.size()==1: # la cola estaba vacía, lo empezamos a atender de inmediato
                tipo_evento="evento_salida" #comienza evento salida
                tiempo_evento=ahora+np.random.exponential(ts) #su tiempo correspondiente
                H.insert((tiempo_evento,tipo_evento)) #se inserta en el Heap
            if nclientes<maxclientes: # programamos la hora de llegada del siguiente cliente
                tipo_evento="evento_llegada" #definimos el evento
                tiempo_evento=ahora+np.random.exponential(ta) #y su hora correspondiente
                H.insert((tiempo_evento,tipo_evento)) #se inserta en el Heap
            if c.is_empty(): #si la cola está vacía entonces el tipo de evento es None
                tipo_evento=None 
        else:               
            # Termina de atenderse un cliente, avanzamos el tiempo simulado a la hora en que sale
            ahora=tiempo_evento
            if debug:
                bitacora.append("T="+str(tiempo_evento)+
                                " Tiempo="+str(tiempo_evento)+" Tipo evento="+str(tipo_evento)+
                                " => SALIDA"+
                                "\n\tcola="+str(c.q))
            tllegada=c.deq() # sacamos al cliente de la cola
            
            acum_tiempo_en_sistema+=(ahora-tllegada)
            if c.is_empty(): # no hay nadie en la cola
                if nclientes==maxclientes: #si el numero de clientes es igual a maxclientes
                  tipo_evento=None #se define el evento como None
                else: #en cualquier otro caso
                 tipo_evento="evento_llegada"  #se define el evento como evento llegada
            else: # Hay un cliente esperando, empezamos a atenderlo
                tipo_evento="evento_salida" #se genera evento salida
                tiempo_evento=ahora+np.random.exponential(ts) #y se le asocia su tiempo correspondiente
                H.insert((tiempo_evento,tipo_evento))
    
    return (acum_largo_cola/maxclientes,acum_tiempo_en_sistema/maxclientes,bitacora)

In [53]:
(cola,demora,bitacora)=simula(10,100,75,debug=True)
print("Largo promedio de la cola=",cola," Tiempo promedio en el sistema=",demora)

Largo promedio de la cola= 1.9  Tiempo promedio en el sistema= 138.02762271192722


In [54]:
print_list(bitacora)

T=21.25986576184801 Tiempo=21.25986576184801 Tipo evento=evento_llegada => LLEGADA
	cola=[]
T=78.83677538497456 Tiempo=78.83677538497456 Tipo evento=evento_llegada => LLEGADA
	cola=[21.25986576184801]
T=94.24603167524747 Tiempo=94.24603167524747 Tipo evento=evento_salida => SALIDA
	cola=[78.83677538497456, 21.25986576184801]
T=207.7973648436669 Tiempo=207.7973648436669 Tipo evento=evento_salida => SALIDA
	cola=[78.83677538497456]
T=232.71542283099382 Tiempo=232.71542283099382 Tipo evento=evento_llegada => LLEGADA
	cola=[]
T=256.5855663702635 Tiempo=256.5855663702635 Tipo evento=evento_salida => SALIDA
	cola=[232.71542283099382]
T=265.07595569718336 Tiempo=265.07595569718336 Tipo evento=evento_llegada => LLEGADA
	cola=[]
T=386.489172340086 Tiempo=386.489172340086 Tipo evento=evento_salida => SALIDA
	cola=[265.07595569718336]
T=582.4168679105603 Tiempo=582.4168679105603 Tipo evento=evento_llegada => LLEGADA
	cola=[]
T=626.7051068662289 Tiempo=626.7051068662289 Tipo evento=evento_llegada 

# Explicación P1

La parte 1 de la tarea corresponde a cambiar un poco la estructura dada usando un Heap.

En un principio se trató de modificar el programa de tal manera que sólo aceptara el Heap, es decir, sin tomar en cuenta la cola, lo cual no se logró por lo que se tuvo que optar por implementar el Heap y la cola, ambas estructuras juntas.

En primer lugar se modificó las estrucutras trepar y hundir, de tal manera que se obtuviese como mejor prioridad el menor tiempo posible. La función trepar se modificó  $a[j][0]<a[(j-1)//2][0]$ que en un principio la comparación era que $a[j]>a[(j-1)//2]$, además se implementó que comparara el primer elemento de la tupla que corresponde al tiempo. En la función hundir se modificó $ k+1<n ,[k+1][0]<a[k][0]$ donde ambos signos de comparaciones eran ">" puesto que en el apunte la mayor prioridad la tenía el mayor número, y en éste caso la mejor prioridad era el menor valor.

Luego se modificó el Heap de tal forma que recibiera tuplas, con la forma dtype="object", y en las comparaciones de tiempo se tomó el primer valor de la tupla (a_insertar[0]).

La metodología a utilizar fue, tal como indicaba la sugerencia antes del while, encolar el primer evento de llegada, cómo se está incorporando el Heap, se insertó un elemento al Heap, éste evento era un evento de llegada y su tiempo, uno exponencial asociado a $t_{a}$. Luego en cada iteración sacar un evento de la cola de prioridad y procesarlo, la manera de extraer un elemento en la cola de prioridad incorporada es extraer mínimo, como éste valor es una tupla, se toma el valor [0] para el tiempo y el valor [1] para saber el tipo de evento. Cuando en el programa original se generaba un evento nuevo y se asignaba a la variable respectiva, ahora hay que insertar un nuevo evento del tipo correspondiente en la cola de prioridad, es decir, en los dos posibles eventos del programa se inserta en el Heap el tiempo correspondiente al evento, es decir, si es evento salida, entonces el tiempo asociado es $t_{s}$ y si es evento de llegada, entonces el tiempo asociado a insertar es $t_{a}$, y como dentro de cada posible evento, hay distintas condiciones para estar en uno o en el otro, entonces dentro de cada condición se evaluó si era necesario o no insertar un elemento o sólo declarar el tipo de evento, ésto es, si en el evento, la cola estaba vacía, entonces el tipo de evento es None, y para el evento de salida, si el número de clientes es distinto al número máximo de clientes, entonces se declara el evento como evento de llegada, esto ya que si se insertaba un elemento, entonces el ciclo no terminaba jamás.

Finalmente, se obtiene lo pedido en la simulación sin cola de prioridad que es lo que se pide, sin embargo se notó una impresición asociada a las variables acum_largo_cola y acum_tiempo_en_sistema ya que ambas corresponden a variables asociadas a la cola, por lo que lo que se obtendría en el resultado sería el valor asociado a la cola y no a la cola de prioridad, sin embargo no se pudo realizar de otra forma.

## 2. Modificar el modelo para introducir un tiempo comprando



In [55]:
import numpy as np
def trepar(a,j): # El elemento a[j] trepa hasta su nivel de prioridad 
    while j>=1 and a[j][0]<a[(j-1)//2][0]: #compara elemento 0 de la tupla, que es el tiempo, pero mueve la tupla completa
        (a[j],a[(j-1)//2])=(a[(j-1)//2],a[j]) # intercambiamos con el padre
        j=(j-1)//2 # subimos al lugar del padre
        
def hundir(a,j,n): # El elemento a[j] se hunde hasta su nivel de prioridad
    while 2*j+1<n: # mientras tenga al menos 1 hijo
        k=2*j+1 # el hijo izquierdo
        if k+1<n and a[k+1][0]<a[k][0]: # el hijo derecho existe y es menor
            k+=1
        if a[j][0]<=a[k][0]: # tiene mejor prioridad que ambos hijos
            break
        (a[j],a[k])=(a[k],a[j]) # se intercambia con el menor de los hijos
        j=k # bajamos al lugar del menor de los hijos
    
class Heap:
    def __init__(self,maxn=100):
        self.a=np.zeros(maxn, dtype="object")
        self.n=0
    def insert(self,x):
        assert self.n<len(self.a)
        self.a[self.n]=x    
        trepar(self.a,self.n)
        self.n+=1       
    def extract_min(self):
        x=self.a[0] # esta variable lleva el mínimo, el casillero 0 queda vacante
        self.n-=1   # achicamos el heap
        self.a[0]=self.a[self.n] # movemos el elemento sobrante hacia el casillero vacante
        hundir(self.a,0,self.n)
        return x
    def imprimir(self):
        print(self.a[0:self.n])
    def size(self): 
        return self.n
    def is_empty(self):
        return self.a is None

class Cola:
    def __init__(self):
        self.q=[]
    def enq(self,x):
        self.q.insert(0,x)
    def deq(self):
        assert len(self.q)>0
        return self.q.pop()
    def is_empty(self):
        return len(self.q)==0
    def size(self):
        return len(self.q)

In [56]:
def print_list(L):
    for x in L:
        print(x)
        
def simula(maxclientes, ta, ts, tc, debug = False):
    # Simula el paso de maxclientes clientes por el sistema, donde
    # ta = tiempo promedio entre llegadas (arrivals)
    # ts = tiempo promedio de servicio
    # maxclientes = número máximo de clientes para la simulación
    # debug = True => genera bitácora y usa secuencia aleatoria reproducible
    # retorna (largo promedio de la cola, tiempo promedio en el sistema)

    if debug: 
        np.random.seed(1234)

    ahora = 0 # Tiempo simulado
    nclientes = 0 # Número de clientes que han llegado
    H = Heap(maxclientes) 
    c = Cola()
    acum_largo_cola = 0 # acumula largo de la cola para sacar promedio al final
    acum_tiempo_en_sistema = 0 # acumula tiempos en el sistema para sacar promedio al final
    bitacora = [] #En caso de que se pida debug
    
    #al igual que en el caso anterior se inserta el primer elemento en el Heap

    tipo_evento = "evento_llegada"
    tiempo_evento = np.random.exponential(ta)
    
    H.insert((tiempo_evento, tipo_evento))
     
    while tipo_evento is not None: #en cada iteracion se ve que tipo_evento no sea None y se procesa, esto es se extrae el minimo y su tiempo y tipo de evento correspondiente

        evento = H.extract_min() 
        ahora = evento[0] 
        tipo_evento = evento[1] 

        #primer evento es el de llegada
        if tipo_evento == "evento_llegada":
            nclientes += 1
           
            if debug: 
                bitacora.append("T = " + str(ahora) +
                                " Tiempo evento=" + str(tiempo_evento)+ " Tipo evento=" + str(tipo_evento)+
                                " => LLEGADA"+
                                "\n\tCola = " + str(c.q))
            #se pasa a evento tipo fin compra
            tipo_evento = "fin_compra"
            tiempo_evento = ahora + np.random.exponential(tc)
            # se inserta en el Heap
            H.insert((tiempo_evento, tipo_evento))
            acum_tiempo_en_sistema += (tiempo_evento - ahora) 
            #bajo cierta condicion, el evento es evento de llegada         
            if nclientes < maxclientes: 
                tipo_evento = "evento_llegada"
                tiempo_evento = ahora + np.random.exponential(ta)
                #se inserta en el Heap
                H.insert((tiempo_evento, tipo_evento))
            #evento agregado llamado fin compra que ocurre bajo ciertas condiciones
        elif tipo_evento == "fin_compra":
            c.enq(ahora) #se encola el tiempo en la cola
            acum_largo_cola += c.size()
            if debug: 
                bitacora.append("T = " + str(ahora) +
                                " Tiempo evento=" + str(tiempo_evento)+ " Tipo evento=" + str(tipo_evento)+
                                " => FIN COMPRA"+
                                "\n\tCola = " + str(c.q))
            if c.size() == 1:
                tipo_evento = "evento_salida"
                tiempo_evento = ahora + np.random.exponential(ts)
                # se inserta el elemento correspondiente en el Heap
                H.insert((tiempo_evento, tipo_evento))
        else:
            
            if debug: 
                bitacora.append("T = " + str(ahora) +
                                " Tiempo evento=" + str(tiempo_evento)+ " Tipo evento=" + str(tipo_evento)+
                                " => SALIDA"+
                                "\n\tCola = " + str(c.q))
            tllegada = c.deq() #se saca el elemento de la cola
            acum_tiempo_en_sistema += (ahora - tllegada) #se acumula en ésta variable
            if c.is_empty(): #si la cola está vacía
                if nclientes < maxclientes: #si el numero de clientes es menor al maximo de clientes
                    tipo_evento = "evento_llegada" #define evento como evento llegada
                if H.size() == 0: #tamaño del heap es cero
                    tipo_evento = None #el evento es None
            else:  #si la cola no está vacía
                tipo_evento = "evento_salida" #define el evento como evento salida
                tiempo_evento = ahora + np.random.exponential(ts) #junto con su hora correspondiente
                #se inserta en el Heap
                H.insert((tiempo_evento, tipo_evento))

    return (acum_largo_cola/maxclientes, acum_tiempo_en_sistema/maxclientes, bitacora)

In [57]:
(cola,demora,bitacora)=simula(10,100,75,300,debug=True)
print("Largo promedio de la cola=",cola," Tiempo promedio en el sistema=",demora)

Largo promedio de la cola= 1.5  Tiempo promedio en el sistema= 417.546881178519


In [58]:
print_list(bitacora)

T = 21.25986576184801 Tiempo evento=21.25986576184801 Tipo evento=evento_llegada => LLEGADA
	Cola = []
T = 78.83677538497456 Tiempo evento=78.83677538497456 Tipo evento=evento_llegada => LLEGADA
	Cola = []
T = 230.23855294286716 Tiempo evento=230.23855294286716 Tipo evento=evento_llegada => LLEGADA
	Cola = []
T = 262.5990858090567 Tiempo evento=262.5990858090567 Tipo evento=evento_llegada => LLEGADA
	Cola = []
T = 313.20452941544585 Tiempo evento=579.9399980224337 Tipo evento=fin_compra => FIN COMPRA
	Cola = [313.20452941544585]
T = 325.719127099946 Tiempo evento=469.724323866816 Tipo evento=fin_compra => FIN COMPRA
	Cola = [325.719127099946, 313.20452941544585]
T = 469.724323866816 Tiempo evento=469.724323866816 Tipo evento=evento_salida => SALIDA
	Cola = [325.719127099946, 313.20452941544585]
T = 502.9405030835674 Tiempo evento=502.9405030835674 Tipo evento=evento_salida => SALIDA
	Cola = [325.719127099946]
T = 540.4727177230324 Tiempo evento=502.9405030835674 Tipo evento=fin_compra 

# Explicación P2

Se creó un nuevo evento, evento fin compra, el cuál, después de llegar al supermercado, la persona, se queda un tiempo exponencial asociado a $t_{c}$ comprando, se definió en el código entre los eventos llegada y salida. Para estar en el evento fin compra, basta con estar situado en el evento llegada, luego de ello se está en el evento fin compra, luego estando en este tipo de evento, si el largo de la cola es igual a 1, entonces cambiamos a evento salida.

Siguiendo la sugerencia de la pregunta anterior, antes del while, encolar el primer evento de llegada, cómo se está incorporando el Heap, se insertó un elemento al Heap, éste evento era un evento de llegada y su tiempo, uno exponencial para  $t_{a}$ . Luego en cada iteración sacar un evento de la cola de prioridad y procesarlo, la manera de extraer un elemento en la cola de prioridad incorporada es extraer mínimo, como éste valor es una tupla, se toma el valor [0] para el tiempo y el valor [1] para saber el tipo de evento. Cuando en el programa original se generaba un evento nuevo y se asignaba a la variable respectiva, ahora hay que insertar un nuevo evento del tipo correspondiente en la cola de prioridad, es decir, en los tres posibles eventos del programa se inserta en el Heap el tiempo correspondiente al evento, es decir, si es evento salida, entonces el tiempo asociado es  $t_{s}$, si es evento de llegada, entonces el tiempo asociado a insertar en  $t_{a}$ y si el evento es fin compra, entonces el tiempo asociado es $t_{c}$. En los eventos llegada y fin compra, en cada una de sus condiciones se insertó el elemento correspondiente en el Heap (tipo de evento y su tiempo correspondiente), para el evento salida, si la cola está vacía, si el tamaño del Heap es cero, entonces el tipo evento es None, si el número de clientes es menor que el número máximo de clientes, entonces se declara el evento como evento llegada, si la cola es distinta de vacío, entonces se inserta en el Heap el elemento evento salida y su tiempo correspondiente.

Finalmente se obtiene que el largo promedio de la cola es 1.5 y el tiempo promedio en el sistema es 417, la bitácora generada introduce el evento fin compra además de los dos eventos declarados anteriormente.